# CLI

> The Plash CLI tool

This module implements the Plash CLI functions.
As they're built with fastcore `@call_parse` they can also be used in python directly.

In [ ]:
#| default_exp cli

In [ ]:
#| export
from fastcore.all import *
from fastcore.xdg import *
import secrets, webbrowser, json, httpx, io, tarfile, random, string
from pathlib import Path
from uuid import uuid4
from time import time, sleep
import io, os, re, tarfile, tomllib

from plash_cli import __version__

In [ ]:
#| hide
from tempfile import TemporaryDirectory
from nbdev.showdoc import show_doc

## Helpers -

In [ ]:
#| export
PLASH_CONFIG_HOME = xdg_config_home() / 'plash_config.json'
PLASH_DOMAIN = os.getenv("PLASH_DOMAIN","pla.sh")  # pla.sh plash-dev.answer.ai localhost:5002

In [ ]:
#| export
def _get_client(cfg=PLASH_CONFIG_HOME):
    client = httpx.Client()
    if tok := os.getenv("PLASH_TOKEN"): cookies = {"session_": tok}
    elif cfg.exists(): cookies = cfg.read_json()
    else: raise FileNotFoundError("Config not found. Run plash_login and retry.")
    client.cookies.update(cookies)
    client.headers.update({'X-PLASH': 'true', 'User-Agent': f'plash_cli/{__version__}'})
    return client

In [ ]:
#| export
def _mk_auth_req(url:str, method:str='get', timeout=300., **kwargs):
    r = getattr(_get_client(), method)(url, timeout=timeout, **kwargs)
    if r.status_code == 200: return r
    else: print(f'Failure: {r.headers["X-Plash-Error"]}')

In [ ]:
#| export
def _get_app_name(path:Path):
    plash_app = Path(path) / '.plash'
    if not plash_app.exists(): raise FileNotFoundError(f"File not found: {plash_app=}")
    env = parse_env(fn=plash_app)
    if name:=env.get("PLASH_APP_NAME"): return name
    if aid :=env.get('PLASH_APP_ID'): 
            plash_app.write_text(f"export PLASH_APP_NAME={aid}")
            return aid
    raise RuntimeError(f"{plash_app=} did not have a PLASH_APP_NAME")


In [ ]:
#| export
def _prep(path,name): return name if name else _get_app_name(Path(path))

In [ ]:
#| export
def _endpoint(sub='', rt=''):
    p = "http" if "localhost" in PLASH_DOMAIN else "https"
    return f"{p}://{sub}{'.' if sub else ''}{PLASH_DOMAIN}{rt}"

In [ ]:
#| export
def _is_included(path):
    "Returns True if path should be included in deployment"
    if path.name.startswith('.'): return False
    if path.suffix == '.pyc': return False
    excludes = {'.git', '__pycache__', '.gitignore', '.env', 
                '.pytest_cache', '.venv', 'venv', '.ipynb_checkpoints',
                '.vscode', '.idea', '.sesskey'}
    return not any(p in excludes for p in path.parts)

In [ ]:
#| export
#| hide
class PlashError(Exception): pass

In [ ]:
#| export
def _poll_cookies(paircode, interval=1, timeout=180):
    "Poll server for token until received or timeout"
    start = time()
    client = httpx.Client()
    url = _endpoint(rt=f"/cli_token?paircode={paircode}")
    while time()-start < timeout:
        resp = client.get(url).raise_for_status()
        if resp.text.strip(): return dict(client.cookies)
        sleep(interval)

In [ ]:
#| export
@call_parse
def login(
    token:str=None,  # Token to save directly to config
    show:bool=False  # Output the current session token
):
    "Authenticate CLI with server and save config"
    if show:
        if not PLASH_CONFIG_HOME.exists(): return print("No config found.")
        return print(PLASH_CONFIG_HOME.read_json().get("session_", ""), end='')
    if token:
        PLASH_CONFIG_HOME.write_text(json.dumps({"session_": token.strip()}))
        return f"Token saved to {PLASH_CONFIG_HOME}"
    paircode = secrets.token_urlsafe(16)
    login_url = httpx.get(_endpoint(rt=f"/cli_login?paircode={paircode}")).text
    print(f"Opening browser for authentication:\n{login_url}\n")
    webbrowser.open(login_url)
    
    cookies = _poll_cookies(paircode)
    if cookies:
        PLASH_CONFIG_HOME.write_text(json.dumps(cookies))
        print(f"Authentication successful! Config saved to {PLASH_CONFIG_HOME}")
    else: print("Authentication timed out.")

CLI usage:

In [ ]:
%%bash
plash_login --help

usage: plash_login [-h] [--token TOKEN] [--show]

Authenticate CLI with server and save config

opti

ons:
  -h, --help     show this help message and exit
  --token TOKEN  Token to save directly to con

fig
  --show         Output the current session token (default: False)


In [ ]:
#| export
pat = r'(?m)^# /// (?P<type>[a-zA-Z0-9-]+)$\s(?P<content>(^#(| .*)$\s)+)^# ///$'

def _deps(script: bytes | str):
    'Get the dependencies from the script. From: https://peps.python.org/pep-0723/'
    name = 'script'
    if isinstance(script, bytes): script = script.decode('utf-8')
    matches = L(re.finditer(pat, script)).filter(lambda m: m.group('type') == name)
    if len(matches) > 1: raise ValueError(f'Multiple {name} blocks found')
    elif len(matches) == 1:
        content = ''.join(line[2:] if line.startswith('# ') else line[1:]
                          for line in matches[0].group('content').splitlines(keepends=True))
        return '\n'.join(tomllib.loads(content)['dependencies'])
    else: return None

In [ ]:
#| hide
# Lets test on some demo apps:
test_eq(_deps(Path("../examples/inline_dependencies/main.py").read_text()),"python-fasthtml")
test_is(_deps(Path("../examples/fasthtml/main.py").read_text()),None)

In [ ]:
#| export
def _validate_app(path):
    "Validates directory `path` is a deployable Plash app"
    if not (path / 'main.py').exists():
        raise PlashError('A Plash app requires a main.py file.')
    deps = _deps((path / 'main.py').read_text(encoding='utf-8'))
    if  deps and (path/"requirements.txt").exists(): 
        raise PlashError('A Plash app should not contain both a requirements.txt file and inline dependencies (see PEP723).')

In [ ]:
#| hide
# All test apps should be valid:
for d in Path("../examples/").iterdir(): test_is(_validate_app(d),None)

In [ ]:
#| hide
# Lets test each failure case:
with TemporaryDirectory() as td:
    td = Path(td)
    test_fail(_validate_app, args=(td,), contains="main.py")
    
    # test failure case of deps in both main.py and requirements.txt
    (td / "requirements.txt").write_text("")
    (td / "main.py").write_text(Path("../examples/inline_dependencies/main.py").read_text())
    test_fail(_validate_app, args=(td,), contains="not contain both")

In [ ]:
#| export
#| hide
def create_tar_archive(path:Path, force_data:bool=False) -> tuple[io.BytesIO, int]:
    "Creates a tar archive of a directory, excluding files based on is_included"
    tarz = io.BytesIO()
    files = L(path if path.is_file() else path.iterdir()).filter(_is_included)
    if not force_data: files = files.filter(lambda f: f.name != 'data')
    with tarfile.open(fileobj=tarz, mode='w:gz') as tar:
        for f in files: tar.add(f, arcname=f.name)
        if deps:=_deps((path/'main.py').read_bytes()):
            info = tarfile.TarInfo('requirements.txt')
            info.size = len(deps)
            tar.addfile(info, io.BytesIO(deps.encode('utf-8')))
    tarz.seek(0)
    return tarz, len(files)

In [ ]:
#| export
def deploy(
    path:Path='.',    # Path to project
    name:str=None,          # Overrides the .plash file in project root if provided
    force_data:bool=False): # Overwrite data/ directory during deployment
    """
    Deploys app to production. By default, this command erases all files in your app which are not in data/.
    Then uploads all files and folders, except paths starting with `.` and except the local data/ directory.
    If `--force_data` is used, then it erases all files in production. Then it uploads all files and folders,
    including `data/`, except paths starting with `.`.
    """
    path = Path(path)
    if name == '': raise PlashError('App name cannot be an empty string')
    if not path.is_dir(): raise PlashError("Path should point to the project directory")
    _validate_app(path)
    
    try: 
        if not name: name = _get_app_name(path)
    except FileNotFoundError:
        plash_app = path / '.plash'
        name = friendly_name(3, 3)
        plash_app.write_text(f'export PLASH_APP_NAME={name}')
    
    tarz, _ = create_tar_archive(path, force_data)
    r = _mk_auth_req(_endpoint(rt="/upload"), "post", files={'file': tarz},
                     data={'name': name, 'force_data': force_data})
    if not r: raise PlashError('Unknown failure')
    return name if "." in name else _endpoint(sub=name)

In [ ]:
#| export
@call_parse
@delegates(deploy)
def _deploy(**kwargs):
    print('Initializing deployment...')
    try: res = deploy(**kwargs)
    except PlashError as e: return str(e)
    print('✅ Upload complete! Your app is currently being built.\n' +
        f'It will be live at {res}')
_deploy.__doc__ = deploy.__doc__

CLI usage:

In [ ]:
%%bash
plash_deploy --help

usage: plash_deploy [-h] [--path PATH] [--name NAME] [--force_data]

options:
  -h, --help    show t

his help message and exit
  --path PATH   Path to project (default: .)
  --name NAME   Overrides the

 .plash file in project root if provided
  --force_data  Overwrite data/ directory during deployment

 (default: False)


In [ ]:
#| export
@call_parse
def view(
    path:Path='.', # Path to project directory
    name:str=None,     # Overrides the .plash file in project root if provided
):
    "Open your app in the browser"
    name = _prep(path, name)
    url = name if '.' in name else _endpoint(sub=name)
    print(f"Opening browser to view app :\n{url}\n")
    webbrowser.open(url)

CLI usage:

In [ ]:
%%bash
plash_view --help

usage: plash_view [-h] [--path PATH] [--name NAME]

Open your app in the browser

options:
  -h, --h

elp   show this help message and exit
  --path PATH  Path to project directory (default: .)
  --name

 NAME  Overrides the .plash file in project root if provided


In [ ]:
#| export
def delete(
    path:Path='.', # Path to project
    name:str=None):      # Overrides the .plash file in project root if provided
    'Delete your deployed app'
    name = _prep(path, name)
    r = _mk_auth_req(_endpoint(rt=f"/delete?name={name}"), "delete")
    if not r: raise PlashError('Failed to delete app')
    return f"App '{name}' deleted successfully"

@call_parse
@delegates(delete)
def _delete(force:bool=False,  # Skip confirmation prompt
    **kwargs):
    'Delete your deployed app'
    if not force:
        confirm = input("Are you sure you want to delete the app? [y/N]: ")
        if confirm.lower() not in ['y', 'yes']: return print("Deletion cancelled.")
    try: print(delete(**kwargs))
    except PlashError as e:  return str(e)
_delete.__doc__ = delete.__doc__

CLI usage:

In [ ]:
%%bash
plash_delete --help

usage: plash_delete [-h] [--path PATH] [--name NAME] [--force]

Delete your deployed app

options:
 

 -h, --help   show this help message and exit
  --path PATH  Path to project (default: .)
  --name N

AME  Overrides the .plash file in project root if provided
  --force      Skip confirmation prompt (

default: False)


In [ ]:
#| export
def start(
    path:Path='.', # Path to project
    name:str=None):      # Overrides the .plash file in project root if provided
    'Start your deployed app'
    name = _prep(path, name)
    r = _mk_auth_req(_endpoint(rt=f"/start?name={name}"))
    if not r: raise PlashError('Failed to start app')
    return f"App '{name}' started"

@call_parse
@delegates(start)
def _start(**kwargs):
    try: print(start(**kwargs))
    except PlashError as e:  return str(e)
_start.__doc__ = start.__doc__

CLI usage:

In [ ]:
%%bash
plash_start --help

usage: plash_start [-h] [--path PATH] [--name NAME]

options:
  -h, --help   show this help message 

and exit
  --path PATH  Path to project (default: .)
  --name NAME  Overrides the .plash file in pro

ject root if provided


In [ ]:
#| export
def stop(
    path:Path='.', # Path to project
    name:str=None):      # Overrides the .plash file in project root if provided
    'Stop your deployed app'
    name = _prep(path, name)
    r = _mk_auth_req(_endpoint(rt=f"/stop?name={name}"))
    if not r: raise PlashError('Failed to stop app')
    return f"App '{name}' stopped"

@call_parse
@delegates(stop)
def _stop(**kwargs):
    try: print(stop(**kwargs))
    except PlashError as e:  return str(e)
_stop.__doc__ = stop.__doc__

CLI usage:

In [ ]:
%%bash
plash_stop --help

usage: plash_stop [-h] [--path PATH] [--name NAME]

options:
  -h, --help   show this help message a

nd exit
  --path PATH  Path to project (default: .)
  --name NAME  Overrides the .plash file in proj

ect root if provided


In [ ]:
#| export
log_modes = str_enum('log_modes', 'build', 'app')

In [ ]:
#| export
def logs(
    path:Path='.',    # Path to project
    name:str=None,          # Overrides the .plash file in project root if provided
    mode:log_modes='build'): # Choose between build or app logs
    'Get logs for your deployed app'
    name = _prep(path, name)
    r = _mk_auth_req(_endpoint(rt=f"/logs?name={name}&mode={mode}"))
    if not r: raise PlashError('Failed to retrieve logs')
    return r.text

@call_parse
@delegates(logs)
def _logs(tail:bool=False,  # Tail the logs
    **kwargs):
    try:
        if not tail: return print(logs(**kwargs))
        text = ''
        while True:
            try:
                new_text = logs(**kwargs)
                print(new_text[len(text):], end='')
                text = new_text
                if kwargs.get('mode','build') == 'build' and 'Build End Time:' in new_text: return
                sleep(1)
            except KeyboardInterrupt: print("\nExiting"); return
    except PlashError as e:  return str(e)
_logs.__doc__ = logs.__doc__

CLI usage:

In [ ]:
%%bash
plash_logs --help

usage: plash_logs [-h] [--path PATH] [--name NAME] [--mode {build,app}] [--tail]

options:
  -h, --h

elp          show this help message and exit
  --path PATH         Path to project (default: .)
  --

name NAME         Overrides the .plash file in project root if provided
  --mode {build,app}  Choose

 between build or app logs (default: build)
  --tail              Tail the logs (default: False)


In [ ]:
#| export
@patch
def _is_dir_empty(self:Path): return next(self.iterdir(), None) is None

In [ ]:
#| hide
import tempfile

In [ ]:
#| hide
temp_dir = tempfile.TemporaryDirectory()
td = Path(temp_dir.name)

test_eq(td._is_dir_empty(), True)
(td/'.temp').write_text('Hello, world!')
test_eq(td._is_dir_empty(), False)

In [ ]:
#| hide
temp_dir.cleanup()

In [ ]:
#| export
def download(
    path:Path='.',  # Path to project
    name:str=None,        # Overrides the .plash file in project root if provided
    save_path:Path="./download/"): # Save path (optional)
    "Download deployed app to save_path."
    name = _prep(path, name)
    save_path = Path(save_path)
    save_path.mkdir(exist_ok=True)
    if not save_path._is_dir_empty(): raise PlashError(f'Save path ({save_path}) is not empty.')
    r = _mk_auth_req(_endpoint(rt=f'/download?name={name}'))
    if not r: raise PlashError('Download request failed')
    with tarfile.open(fileobj=io.BytesIO(r.content), mode="r:gz") as tar: 
        tar.extractall(path=save_path, filter='data')
    return save_path

@call_parse
@delegates(download)
def _download(**kwargs):
    try: print(f"Downloaded your app to: {download(**kwargs)}")
    except PlashError as e:  return str(e)
_download.__doc__ = download.__doc__

CLI usage:

In [ ]:
%%bash
plash_download --help

usage: plash_download [-h] [--path PATH] [--name NAME] [--save_path SAVE_PATH]

options:
  -h, --hel

p             show this help message and exit
  --path PATH            Path to project (default: .)


  --name NAME            Overrides the .plash file in project root if provided
  --save_path SAVE_PA

TH  Save path (optional) (default: ./download/)


In [ ]:
#| export
def app_list():
    "List your deployed apps"
    r = _mk_auth_req(_endpoint(rt="/user_apps"))
    if not r: raise PlashError('Failed to retrieve')
    return r.json()

@call_parse
def _app_list(verbose:bool=False): # Whether to show running status as well as name: 1=running, 0=stopped
    try: res = apps()
    except PlashError as e: return str(e)
    if not res: print("You don't have any deployed Plash apps.")
    for a in res: print(f"{a['running']} {a['name']}" if verbose else a['name'])
_app_list.__doc__ = app_list.__doc__

CLI usage:

In [ ]:
%%bash
plash_apps --help

usage: plash_apps [-h] [--verbose]

options:
  -h, --help  show this help message and exit
  --verbo

se   Whether to show running status as well as name: 1=running, 0=stopped (default: False)


In [ ]:
#| export
def plash_tool_info():
    from dialoghelper import add_msg
    add_msg('Plash tools: &`[login, deploy, delete, start, stop, logs, download, app_list]`')

## Export -

In [ ]:
#| hide
#| eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()